# Clojupyter demo: Jupyter Notebook-only features

This notebook demonstrates some features of Clojupyter for Jupyter Notebook. Please note that it uses <font color=red>**Notebook-only functionality**</font> and does not render correctly using *Jupyter Lab* which uses renderers instead of allowing direct access to external Javascript libraries.  See the Jupyter Lab demo notebook for details.


## Charting using  external Javascript library: Highcharts

Since you can render arbitrary HTML using display/hiccup-html, it's pretty easy to use external Javascript libraries to do things like generate charts. Here's an example using Highcharts.

In [ ]:
(require '[clojupyter.javascript.alpha :as cjp-js])
(require '[clojupyter.display :as display])
(require '[clojupyter.misc.helper :as helper])
(require '[clojure.data.json :as json])
(cjp-js/amd-add-javascript-html
  [{:ident :hicharts :exports "Highcharts" :url "https://code.highcharts.com/highcharts"}])

In [ ]:
(def raw-data (map #(int (+ (* 22 (+ % (Math/random)) 78))) (range)))
(def data-1 (take 25 raw-data))
(def data-2 (take 25 (drop 25 raw-data)))

In [ ]:
(defn highchart-chart
  [nodeid data]
  (cjp-js/amd-wrap-require [:hicharts]
    (format "function(hc){hc.chart('%s',%s)}" nodeid (json/write-str data))))

In [ ]:
(def CHART {:chart {:type "line"}
            :title {:text (str "External js library: Highcharts plot using " (:formatted-version clojupyter/*version*))}
            :series [{:data data-1} {:data data-2}]})

In [ ]:
(display/hiccup-html 
 (let [nodeid (gensym)]
   [:div 
    [:div {:id nodeid}]
    [:script (highchart-chart nodeid CHART)]]))

<font color=red>**OBS The cell above fails to render the resulting Vega graph.  Needs fixing.**</font>

## Charting using Clojure visualization library Oz

[Oz](https://github.com/metasoarous/oz) is a data visualization and scientific document processing library for Clojure built around [Vega Lite](https://vega.github.io/vega-lite/) & [Vega](https://vega.github.io/).  Oz provides has a built-in interface to clojupyter, so all we have to do to use it is to load it into Clojure:

In [ ]:
(helper/add-dependencies '[metasoarous/oz "1.5.6"])
(require '[oz.notebook.clojupyter :as oz]);

For this demo we define some simple functions generating some data compatible with the high-level  charting library:

In [ ]:
(defn datapt [[s n]] {:Index s, :Value n})
(defn graph  [vs]    {:data {:values vs}, 
                      :mark :line
                      :encoding {:x {:field :Index}
                                 :y {:field :Value}}})
(def graph-it (comp graph (partial map datapt) (partial apply map vector)));

With a generator for (slightly) random data...

In [ ]:
(defn raw-data [] (map #(+ % (rand-int 25)) (range)))

...we can create a graph using Vega Lite:

In [ ]:
(->> [(range) (take 150 (raw-data))]
     graph-it
     oz/view!)

**Note**: The above cell is *not* supposed to render correctly in Jupyter Lab, <font color=red>**only Jupyter Notebook**</font>.  See Jupyter Lab demo notebook for details on how accomplish the same thing in Jupyter Lab.